# Goal

The goal of this notebook is to demonstrate the usage of the new, multi-objective, and interactive version of the Tomograph.

## Goals for the Tomograph2

1. Move away from matplotlib to either plotly, or bokeh. 
2. Make sure that:
    1. All scales are the same so that they are comparable.
    2. We should be able to plot not just predictions but also individual observations as a scatterplot. 
    3. We should be able to link the points across multiple charts to investigate them more deeply (bokeh seems to be the winner here)
    4. We should be able to connect multiple tomographs to the same optimizer, as any given analysis might focus on different aspects and each aspect should have it's own graphs.
3. The Tomograph produces a set of heatmaps that belong to 2D cross-sections of the higher-dimensional hypercube. All these cross-sections share a single point (often the optimum). The user should be able to:
    1. Select one of the predefined points (different types of optima, maybe optima for different contexts)
    2. View the resulting cross sections and sensitivity analysis for such a point
    3. Be able to use sliders to adjust the point so that they can easily "walk the hypercube"
    

Plotly has the advantage of also enabling 3D visualizations of the pareto frontier and generates surface plots etc. But bokeh has built-in support for linking data. 

Given what we need, we will start with bokeh, for dabl-style and tomograph plots, and then switch to plotly for parameter interactions, and 3D pareto visualizations.

https://docs.bokeh.org/en/latest/index.html

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Create and train the Optimizer

In [ ]:
from mlos.OptimizerEvaluationTools.ObjectiveFunctionFactory import ObjectiveFunctionFactory, objective_function_config_store

#objective_function_config = objective_function_config_store.get_config_by_name("multi_objective_2_mutually_exclusive_polynomials")
#objective_function_config = objective_function_config_store.get_config_by_name("three_level_quadratic")
#objective_function_config = objective_function_config_store.get_config_by_name("2d_hypersphere_minimize_none")
objective_function_config = objective_function_config_store.get_config_by_name("multi_objective_waves_3_params_2_objectives_half_pi_phase_difference")

objective_function = ObjectiveFunctionFactory.create_objective_function(objective_function_config)
optimization_problem = objective_function.default_optimization_problem

In [ ]:
from mlos.Optimizers.BayesianOptimizerFactory import BayesianOptimizerFactory, bayesian_optimizer_config_store
optimizer_factory = BayesianOptimizerFactory()
optimizer_config = bayesian_optimizer_config_store.default
optimizer = optimizer_factory.create_local_optimizer(
    optimizer_config=optimizer_config,
    optimization_problem=optimization_problem
)

In [ ]:
params_df = objective_function.parameter_space.random_dataframe(num_samples=2000)
objectives_df = objective_function.evaluate_dataframe(params_df)
optimizer.register(parameter_values_pandas_frame=params_df, target_values_pandas_frame=objectives_df)

# Create and plot the original Tomograph

# Plot observations using ModelTomograph2

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.layouts import column
from mlos.OptimizerMonitoring.Tomograph.ModelTomograph2 import ModelTomograph2    
    
        
tomograph2 = ModelTomograph2(optimizer=optimizer)
observations_plot = tomograph2.get_observations_plot()
objectives_plot = tomograph2.get_objectives_plot()
all_plots = column([observations_plot, objectives_plot])
output_notebook()
show(all_plots)

In [ ]:
from bokeh.models.widgets import Tabs, Panel

observations_tab = Panel(child=observations_plot, title="Observations")
objectives_tab = Panel(child=objectives_plot, title="Objectives")
tabs = Tabs(tabs=[observations_tab, objectives_tab])
show(tabs)